## Problem Definition & Objective

### Problem Statement: 
Manual film editing is time-consuming and often ignores the psychological **"Kuleshov Effect"** where the meaning of a shot changes based on the context of the shots surrounding it.

### Objective: 
To algorithmically construct emotionally expressive video montages using rule-based cinematic logic, demonstrating how AI-assisted tools can augment creative storytelling rather than replace it.

## Selected Project Track: AI in Film Editing 

This project lies at the intersection of:
- Artificial Intelligence
- Computational Creativity
- Film & Media Technology

It focuses on AI-assisted creative systems rather than predictive modeling, emphasizing design logic, emotional representation, and responsible automation.


## Real-world Relevance: 
This tool allows creators to rapidly prototype emotional subtexts, using heuristic AI to automate complex editorial structures like "Anxious" pulsing or "Flashback" erasures.

Emotion-driven editing is widely used in:
- Film trailers
- Mental health storytelling
- Artistic short films
- Social impact media

Independent creators often lack access to advanced post-production tools. This system demonstrates how lightweight AI logic can help creators express complex emotional states programmatically, using accessible libraries and transparent logic.


## Data Understanding & Preparation

### Input Data
- Three short video clips representing narrative fragments
- An user-selected emotional state (`mood`) either by choosing from a dropdown list or manual commands.

### Assumptions
- All clips share compatible resolution or are composited
- Emotion is represented symbolically via editing rhythm, pacing, and repetition
- No facial recognition or biometric data is used

## Libraries Used

- Python 3.x
- MoviePy
- NumPy
- FFmpeg (backend)

This notebook assumes all dependencies are installed locally.

## Imports & Setup

In [1]:
import numpy as np
import re

from moviepy.editor import (
    VideoFileClip,
    concatenate_videoclips,
    ColorClip
)

import moviepy.video.fx.all as vfx
from scipy.ndimage import gaussian_filter
from IPython.display import Video

## Model / System Design

This system follows a **rule-based emotional montage architecture**:

- **Anxious** → Rapid interruptions with black frames (visual pulse)
- **Flashback** → Forward playback followed by accelerated reverse
- **Neutral** → Smooth linear continuity

The "model" here is not a neural network but a **deterministic creative system**, where cinematic rules encode emotional intent.


## Logic 

In [2]:
# ---------------- PAGE SETUP ----------------
st.set_page_config(page_title="Lev AI | Meaning Between Cuts", layout="wide")
st.title("🎬 Meaning Between Cuts")
st.subheader("AI Montage Tool by Deepikha Bharadwaj")

# ---------------- INPUT SECTION ----------------
c1, c2, c3 = st.columns(3)
with c1: f1 = st.file_uploader("Neutral Shot", type=["mp4", "mov"])
with c2: f2 = st.file_uploader("Context Shot 1", type=["mp4", "mov"])
with c3: f3 = st.file_uploader("Context Shot 2", type=["mp4", "mov"])

mood = st.selectbox("Narrative Mood", ["Peaceful", "Flashback", "Anxious", "Haze", "Noir", "None"])

manual_enabled = (mood == "None")
if manual_enabled:
    p_in = st.text_input("Manual Effects (e.g., zoom 10, bw all, grain second, reverse)")
else:
    st.info(f"Active Mode: {mood}")
    p_in = ""

# ---------------- ROBUST EFFECT FUNCTIONS ----------------
def grayscale_math(get_frame, t):
    frame = get_frame(t)
    gray = np.dot(frame[..., :3], [0.299, 0.587, 0.114])
    return np.stack([gray]*3, axis=-1).astype("uint8")

def haze_math(get_frame, t):
    frame = get_frame(t)
    blurred = np.zeros_like(frame)
    for i in range(3):
        blurred[:, :, i] = gaussian_filter(frame[:, :, i], sigma=8)
    return blurred.astype("uint8")

# ---------------- MAIN PIPELINE ----------------
if st.button("GENERATE CINEMATIC MONTAGE"):
    if not (f1 and f2 and f3):
        st.error("Please upload all three shots.")
    else:
        with st.spinner(f"Processing {mood} Montage..."):
            try:
                # 1. Clean up old files to prevent conflicts
                for old_file in ["out.mp4", "a.mp4", "b.mp4", "c.mp4"]:
                    if os.path.exists(old_file): os.remove(old_file)

                # 2. Save new uploads
                for name, f in zip(["a", "b", "c"], [f1, f2, f3]):
                    with open(f"{name}.mp4", "wb") as out: out.write(f.getbuffer())

                # 3. Load & Process Clips
                v1 = VideoFileClip("a.mp4").resized(height=1080)
                v2 = VideoFileClip("b.mp4").resized(height=1080)
                v3 = VideoFileClip("c.mp4").resized(height=1080)

                def process(clip, tag):
                    if mood == "Noir": clip = clip.transform(grayscale_math)
                    if mood == "Haze": clip = clip.transform(haze_math)
                    if manual_enabled and p_in:
                        p = p_in.lower()
                        if tag in p or "all" in p:
                            if "bw" in p: clip = clip.transform(grayscale_math)
                            if "reverse" in p: clip = clip.with_effects([vfx.TimeMirror()])
                            if "zoom" in p:
                                match = re.search(r"zoom (\d+)", p)
                                z = float(match.group(1))/100 if match else 0.1
                                clip = clip.resized(lambda t: 1 + z*t)
                    return clip

                s1, s2, s3 = process(v1, "first"), process(v2, "second"), process(v3, "third")

                # 4. ASSEMBLY LOGIC FOR ALL MODES
                if mood == "Anxious":
                    black = ColorClip(size=v1.size, color=(0,0,0), duration=0.25)
                    content = concatenate_videoclips([s1, black, s2, black, s3], method="compose")
                elif mood == "Flashback":
                    fwd = concatenate_videoclips([s1, s2, s3], method="compose")
                    rev = fwd.with_effects([vfx.TimeMirror(), vfx.MultiplySpeed(4.0)])
                    content = concatenate_videoclips([fwd, rev], method="compose")
                else:
                    # Peaceful, Haze, Noir, and None all use smooth transitions
                    content = concatenate_videoclips([s1, s2, s3], method="compose", padding=-0.5)

                # 5. SECURE EXPORT
                # Explicitly naming a new file each time helps avoid browser cache issues
                output_name = f"montage_{int(time.time())}.mp4"
                content.write_videofile(output_name, codec="libx264", audio_codec="aac", fps=24, logger=None)

                # 6. FORCE RELEASE FILE
                v1.close(); v2.close(); v3.close(); content.close()
                time.sleep(1) # Short pause to ensure the OS releases the file

                # 7. DISPLAY & DOWNLOAD
                st.video(output_name)
                with open(output_name, "rb") as f:
                    st.download_button("📥 Download Montage", f, file_name="my_cinematic_edit.mp4")
                st.success(f"{mood} sequence generated successfully!")

            except Exception as e:
                st.error(f"Render Error: {e}")

'cinematic_montage.mp4'

## Evaluation & Analysis

### Qualitative Evaluation
- Emotional intent is communicated through pacing and repetition
- Anxiety is reinforced via visual interruption
- Flashbacks simulate memory distortion

### Limitations
- Emotion selection is manual
- No learning or personalization
- Limited clip diversity

Despite this, the system succeeds as a proof-of-concept for emotionally aware creative automation.


## Ethical Considerations & Responsible AI

- No personal, biometric, or sensitive data is used
- The system avoids emotion inference or psychological diagnosis
- Emotions are artist-defined, not user-profiled
- Creative control remains fully with the human creator

This aligns with Responsible AI principles of transparency, agency, and non-exploitation.


## Conclusion & Future Scope

This project demonstrates how AI-assisted systems can support emotional storytelling without replacing human creativity.

### Future Enhancements
- Emotion detection from audio/music
- ML-based pacing optimization
- Integration with mobile filmmaking tools
- Expanded emotional vocabulary

The system positions AI as a creative collaborator, not an author.
